In [ ]:

!pip install sentence_transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model_name = "google-t5/t5-3b"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentence_model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
quote_string = "Quote"
tag = "Tag"
meaning = "Interpretation"
new_inter = "New"

class QuoteInterpreter():

    def __init__(self, model, tokenizer,  sentence_model) -> None:
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = tokenizer
        self.model = model.to(self.device)
        self.sentence_model = sentence_model.to(self.device)
        self.model.eval()
        self.sentence_model.eval()

    def generate_embeddings(self, data):
        embeds = {}
        for idx, row in data.iterrows():
            with torch.no_grad():
                embedding = self.sentence_model.encode(row[quote_string], convert_to_tensor=True).to(self.device)
            embeds[idx] = embedding
        return embeds

    def prompt(self, data,n, method="zero_shot"):
        if method == "few_shot":
            self.embeddings = self.generate_embeddings(data)

        for i, _ in data.iterrows():
            prompt = self.get_prompt(i, data, method, n)
            data.loc[i, method] = prompt
            with torch.no_grad():
                input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids.to(self.device)
                outputs = self.model.generate(
                    input_ids,
                    min_new_tokens=20,
                    num_beams=5,
                    max_new_tokens=250,
                    num_return_sequences=3,
                    no_repeat_ngram_size=2)
            for idx, beam_output in enumerate(outputs):
                sent = self.tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                data.loc[i, new_inter + str(idx)] = sent
        return data

    def get_prompt(self, quote_idx, data, method, n):
        if method == "cot":
            return self.cot_prompt(data.loc[quote_idx, quote_string], data.loc[quote_idx, tag])

        if method == "few_shot":
            example_idx = self.get_examples(quote_idx, data, n=n)
            return self.few_shot_prompt(data.loc[quote_idx, quote_string], data, examples=example_idx)

        return self.zero_shot_prompt(data.loc[quote_idx, quote_string])

    def zero_shot_prompt(self, quote):
        return f"Explain the meaning of Quote in 1-2 sentences \n  Quote: {quote}\n  Answer:"

    def get_examples(self, quote_idx, df, n):
        quote_embed = self.embeddings[quote_idx]
        similar_quotes = df[df[tag] == df.loc[quote_idx, tag]]
        similar_quotes = similar_quotes.drop(quote_idx)
        similar_quotes = [(idx, self.embeddings[idx]) for idx, _ in similar_quotes.iterrows()]
        cos_sims = {}
        for i in similar_quotes:
            cos_sims[i[0]] = float(util.cos_sim(quote_embed, i[1])[0][0].cpu())
        top_n = sorted(cos_sims.items(), key=lambda x: x[1], reverse=True)[:n]
        return [i[0] for i in top_n]

    def few_shot_prompt(self, quote, df, examples=None):
        ext_prompt = ""
        if examples is not None:
          for i in examples:
            e_quote, e_meaning = df.loc[i, quote_string], df.loc[i, meaning]
            ext_prompt += f'''"Quote: {e_quote}" => "{e_meaning}"\n'''

        return f'''explain the meaning of the quote in 1-2 sentences\n {ext_prompt} "{quote}" => '''


In [ ]:
interpretation_data = pd.read_excel("/content/drive/MyDrive/NLP/final_interpretations.xlsx")
interpretation_data

,Quote,Author,Tag,Interpretation
0,happiness is part of who we are. joy is the feeling,tony deliso,happiness,"The quote suggests that happiness is a fundamental aspect of our identity or inner nature, inherently part of our being. In contrast, joy is described as a temporary emotion or feeling that arises in response to specific events or experiences, highlighting the distinction between a stable state of contentment (happiness) and fleeting moments of intense pleasure (joy)."
1,"you be good. that's your business. the rest, leave to god. that's his business.",fakeer ishavardas,god,"This quote emphasizes the importance of focusing on being a good person and conducting oneself ethically, while entrusting matters that are beyond one's control to a higher power or divine plan, suggesting a division of responsibility between human efforts and spiritual faith."
2,"here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust. if he can't lay this one stupid brick down, you ain't never gonna have a house baby, and it's cold outside.",greg behrendt,truth,"The quote emphasizes the importance of keeping promises, even small ones like calling when expected, as a fundamental aspect of building a relationship based on love and trust. It suggests that failing to follow through on such basic commitments is a warning sign of unreliability and potential relationship failure."
3,"i've found that luck is quite predictable. if you want more luck, take more chances, be more active, show up more often.",brian tracy,wisdom,"This quote suggests that ""luck"" is not merely a result of random chance, but rather the outcome of taking more actions or risks. By engaging more frequently and actively in various activities, a person increases their opportunity to experience positive outcomes, thereby appearing to be ""luckier."""
4,"if everyone is against you, but god is for you. it is more than enough to keep moving.",lailah gifty akita,god,"This quote suggests that having God's support is sufficient, and more powerful than the opposition of everyone else, providing one with the strength and confidence needed to persevere through challenges."
...,...,...,...,...
389,there is nothing in the world so irresistibly contagious as laughter and good humor.,"charles dickens, a christmas carol",humor,"This quote celebrates the infectious nature of laughter and good humor, highlighting their ability to uplift spirits and bring people together. It suggests that laughter has a powerful and contagious effect, spreading joy and positivity among those who share in it. The quote underscores the universal appeal of humor as a source of connection and happiness, emphasizing its ability to transcend barriers and unite people across cultures and backgrounds."
390,you never fail until you stop trying,albert einstein,inspirational,"This quote emphasizes the importance of perseverance and resilience in the face of challenges. It suggests that failure only occurs when one gives up or stops trying. As long as one continues to make efforts and pursue their goals, there is always the potential for success. The quote underscores the value of persistence and determination in overcoming obstacles and achieving one's aspirations."
391,hope! of all ills that men endure the only cheap and universal cure.,abraham cowley,hope,"This quote reflects on the power of hope as a remedy for the various hardships and challenges that individuals face. It suggests that hope is a universally accessible and affordable antidote to the many difficulties of life. By instilling optimism and resilience, hope can provide comfort and strength in times of adversity. The quote underscores the transformative potential of hope in inspiring positive change and overcoming obstacles, highlighting its essential role in the human experience."
392,be the reason someone smiles. be the reason someone feels loved and believes in the goodnes

In [ ]:

interpreter = QuoteInterpreter(model, tokenizer, sentence_model)

In [ ]:
df = interpreter.prompt(interpretation_data, method="zero_shot")

In [ ]:
df.to_excel("/content/drive/MyDrive/NLP/t5-zero-shot.xlsx")

,Quote,Author,Tag,Interpretation,zero_shot,New0,New1,New2
0,happiness is part of who we are. joy is the feeling,tony deliso,happiness,"The quote suggests that happiness is a fundamental aspect of our identity or inner nature, inherently part of our being. In contrast, joy is described as a temporary emotion or feeling that arises in response to specific events or experiences, highlighting the distinction between a stable state of contentment (happiness) and fleeting moments of intense pleasure (joy).",Explain the meaning of Quote in 1-2 sentences \n Quote: happiness is part of who we are. joy is the feeling\n Answer:,Explain the meaning of Quote in 1-2 sentences Quote: happiness is part of who we are. joy is the feeling.,Explain the meaning of Quote in 1-2 sentences Quote: happiness is part of who we are. joy is the feeling Question:,Explain the meaning of Quote in 1-2 sentences Quote: happiness is part of who we are. joy is the feeling Answer:
1,"you be good. that's your business. the rest, leave to god. that's his business.",fakeer ishavardas,god,"This quote emphasizes the importance of focusing on being a good person and conducting oneself ethically, while entrusting matters that are beyond one's control to a higher power or divine plan, suggesting a division of responsibility between human efforts and spiritual faith.","Explain the meaning of Quote in 1-2 sentences \n Quote: you be good. that's your business. the rest, leave to god. that's his business.\n Answer:",": in 1-2 sentences Quote: you be good. that's your business. the rest, leave to god. .... .","in 1-2 sentences Quote: you be good. that's your business. the rest, leave to god.. .... .",": in 1-2 sentences Quote: you be good. that's your business. the rest, leave to god. :... ."
2,"here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust. if he can't lay this one stupid brick down, you ain't never gonna have a house baby, and it's cold outside.",greg behrendt,truth,"The quote emphasizes the importance of keeping promises, even small ones like calling when expected, as a fundamental aspect of building a relationship based on love and trust. It suggests that failing to follow through on such basic commitments is a warning sign of unreliability and potential relationship failure.","Explain the meaning of Quote in 1-2 sentences \n Quote: here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust. if he can't lay this one stupid brick down, you ain't never gonna have a house baby, and it's cold outside.\n Answer:",Quote: here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust.,Explain the meaning of Quote in 1-2 sentences Quote: here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust.,Quote in 1-2 sentences Quote: here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust.
3,"i've found that luck is quite predictable. if you want more luck, take more chances, be more active, show up more often.",brian tracy,wisdom,"This quote suggests that ""luck"" is not merely a result of random chance, but rather the outcome of taking more actions or risks. By engaging more frequently and actively in various activities, a person increases their opportunity to experience positive outcomes, thereby appearing to be ""luckier.""","Explain the meaning of Quote in 1-2 sentences \n Quote: i've found that luck is quite predictable. if you want more luck, take more chances, be more active, show up more often.\n Answer:","Answer: i've found that luck is quite predictable. if you want more luck, take more chances, be more active, show up more often.","

In [ ]:
interpreter = QuoteInterpreter(model, tokenizer, sentence_model)

('be the reason someone smiles. be the reason someone feels loved and believes in the goodness in people.',
 '. someone smiles. be the reason someone feels loved and believes in the goodness of people. A:...... in 1-2 sentences Quote:.. B:. b: be.')

In [ ]:

df = interpreter.prompt(interpretation_data, method="few_shot", n=3)

In [ ]:
# pd.set_option('display.max_colwidth', None)
df.to_excel("/content/drive/MyDrive/NLP/t5-few-3-shot.xlsx")

0                                                                           .. someone in your arms and knowing you hold the whole world..      "this quote suggests that happiness is part of who we are and joy is the feeling, highlighting the importance of inner contentment over external factors."................ "joy is a feeling that arises from happiness." "Quote "
1                                                                                                             "quotote: god deals us all different hands. how we play 'em is up to us." => "this quote emphasizes personal agency and resilience in the face of life's varied challenges, highlighting the importance of individual choices and actions in shaping our destinies."
2    "here's something else to think about: calling when you say you're going to is the very first brick in the house you are building of love and trust." => "this quote emphasizes the importance of communication and commitment in a relationship. it suggests

In [ ]:

prompt = '''Explain the meaning of the quote -
Quote : "a smile is what makes a face beautiful."
Answer : "This quote implies that a smile enhances one's facial attractiveness more than any other physical feature, suggesting that genuine joy and warmth are key components of beauty."

Quote : "sometimes there a hundred lies behind a smile and not a single truth behind a tear."
Answer :  "This quote suggests that smiles can often be used to hide true feelings or dishonest intentions, portraying a facade of happiness or contentment. In contrast, tears are depicted as more genuine expressions, less likely to be used to conceal the truth, indicating raw, unfiltered emotion."

Quote : "peace begins with a smile"
Answer :  '''

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(
    input_ids,
    min_new_tokens=20,
    num_beams=5,
    max_length=200,
    num_return_sequences=3,
    no_repeat_ngram_size=2)

for idx, beam_output in enumerate(outputs):
    sent = tokenizer.decode(beam_output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(f"Beam {idx + 1}: {sent}")


Beam 1:  :       "peace begins with a smile, implying that genuine joy and warmth are key components of peace." Explain the meaning of the quote - Quote ""
Beam 2:  :       "peace begins with a smile, implying that genuine joy and warmth are key components of peace." Explain the meaning of the quote -
Beam 3:  :       "peace begins with a smile, implying that genuine joy and warmth are key components of peace." Explain the meaning of the quote - ""
